In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/textdb3/fake_or_real_news.csv


In [2]:
data = pd.read_csv("/kaggle/input/textdb3/fake_or_real_news.csv")
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
data.shape

(6335, 4)

In [4]:
data = data.dropna()

In [5]:
data.shape

(6335, 4)

In [6]:
data.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [7]:
data.label

0       FAKE
1       FAKE
2       REAL
3       FAKE
4       REAL
        ... 
6330    REAL
6331    FAKE
6332    FAKE
6333    REAL
6334    REAL
Name: label, Length: 6335, dtype: object

In [8]:
data.label=pd.get_dummies(data['label'])['FAKE']

In [9]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0


In [10]:
x = data.title
y = data.label
y

0       1
1       1
2       0
3       1
4       0
       ..
6330    0
6331    1
6332    1
6333    0
6334    0
Name: label, Length: 6335, dtype: uint8

In [11]:
x

0                            You Can Smell Hillary’s Fear
1       Watch The Exact Moment Paul Ryan Committed Pol...
2             Kerry to go to Paris in gesture of sympathy
3       Bernie supporters on Twitter erupt in anger ag...
4        The Battle of New York: Why This Primary Matters
                              ...                        
6330    State Department says it can't find emails fro...
6331    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332    Anti-Trump Protesters Are Tools of the Oligarc...
6333    In Ethiopia, Obama seeks progress on peace, se...
6334    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: title, Length: 6335, dtype: object

In [12]:
from tensorflow.keras.preprocessing.text import one_hot

In [13]:
voc_size = 10000

In [14]:
onehot_repre = [one_hot(words, voc_size) for words in x]
onehot_repre[0]

[8339, 2181, 2548, 9679, 9814]

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import numpy as np

In [16]:
sent_len = 50
embedded_docs = pad_sequences(onehot_repre, padding='pre', maxlen=sent_len)
print(embedded_docs)

[[   0    0    0 ... 2548 9679 9814]
 [   0    0    0 ... 9654 2348 3438]
 [   0    0    0 ... 4218  309 5134]
 ...
 [   0    0    0 ... 5809 3005  831]
 [   0    0    0 ... 6822 3815 3162]
 [   0    0    0 ... 6987 8559 2173]]


In [17]:
model=Sequential()
model.add(Embedding(voc_size, 10, input_length = sent_len))
model.compile('adam', 'mse')

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 10)            100000    
Total params: 100,000
Trainable params: 100,000
Non-trainable params: 0
_________________________________________________________________


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embedded_docs, y, test_size=0.2)

In [20]:
X_train

array([[   0,    0,    0, ..., 6288, 2370, 3438],
       [   0,    0,    0, ..., 4191, 1925, 2391],
       [   0,    0,    0, ..., 8268, 8284, 8088],
       ...,
       [   0,    0,    0, ..., 5642, 7942, 7063],
       [   0,    0,    0, ..., 2111, 2948, 9981],
       [   0,    0,    0, ..., 9888, 3364, 3438]], dtype=int32)

In [21]:
y_train

4482    1
4882    0
4025    0
6255    1
1658    1
       ..
1564    0
1571    0
3143    0
2589    0
5077    0
Name: label, Length: 5068, dtype: uint8

In [22]:
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [23]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [24]:
embedding_vector_features = 40
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(voc_size, embedding_vector_features, input_length=sent_len))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(100))

# Add a Dense layer with 10 units.
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 40)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(
    loss='binary_crossentropy',
    optimizer="adam",
    metrics=["accuracy"],
)


model.fit(
    X_train, y_train, validation_data=(X_test, y_test), batch_size=64, epochs=10
)

Epoch 1/10
80/80 [==============================] - 9s 86ms/step - loss: 0.6711 - accuracy: 0.5867 - val_loss: 0.4811 - val_accuracy: 0.7798
Epoch 2/10
80/80 [==============================] - 6s 73ms/step - loss: 0.7743 - accuracy: 0.8087 - val_loss: 0.4722 - val_accuracy: 0.7695
Epoch 3/10
80/80 [==============================] - 6s 71ms/step - loss: 0.2840 - accuracy: 0.8995 - val_loss: 0.5217 - val_accuracy: 0.7687
Epoch 4/10
80/80 [==============================] - 6s 71ms/step - loss: 0.1881 - accuracy: 0.9477 - val_loss: 0.5246 - val_accuracy: 0.7766
Epoch 5/10
80/80 [==============================] - 6s 70ms/step - loss: 0.1174 - accuracy: 0.9666 - val_loss: 0.6068 - val_accuracy: 0.7751
Epoch 6/10
80/80 [==============================] - 6s 70ms/step - loss: 0.0810 - accuracy: 0.9764 - val_loss: 0.6951 - val_accuracy: 0.7680
Epoch 7/10
80/80 [==============================] - 6s 72ms/step - loss: 0.0515 - accuracy: 0.9875 - val_loss: 0.7881 - val_accuracy: 0.7656
Epoch 8/10
80

In [26]:
y_pred=model.predict(X_test)
y_pred

array([[2.7447087e-01],
       [9.9679536e-01],
       [6.4467192e-03],
       ...,
       [9.9125969e-01],
       [5.5753095e-05],
       [9.9009657e-01]], dtype=float32)

In [27]:
import numpy as np
y_classes = [np.argmax(element) for element in y_pred]
y_classes

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
